# EDA for Mental Health in Tech Survey data

https://www.kaggle.com/osmi/mental-health-in-tech-survey

In [1]:
import altair as alt
import pandas as pd
data = pd.read_csv('../data/raw/survey.csv')

# Reference Data

In [2]:

states = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DC":"Washington DC", "DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}

In [3]:
data['state_fullname'] =  data['state'].map(states)
data

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments,state_fullname
0,2014-08-27 11:29,37,Female,United States,IL,NaN,No,Yes,Often,25-Jun,...,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN,Illinois
1,2014-08-27 11:29,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Maybe,No,No,No,No,No,Don't know,No,NaN,Indiana
2,2014-08-27 11:29,32,Male,Canada,NaN,NaN,No,No,Rarely,25-Jun,...,No,No,Yes,Yes,Yes,Yes,No,No,NaN,NaN
3,2014-08-27 11:29,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN,NaN
4,2014-08-27 11:30,31,Male,United States,TX,NaN,No,No,Never,100-500,...,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN,Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17,26,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,...,No,No,Some of them,Some of them,No,No,Don't know,No,NaN,NaN
1255,2015-09-26 1:07,32,Male,United States,IL,No,Yes,Yes,Often,26-100,...,No,No,Some of them,Yes,No,No,Yes,No,NaN,Illinois
1256,2015-11-07 12:36,34,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,...,Yes,Yes,No,No,No,No,No,No,NaN,California
1257,2015-11-30 21:25,46,f,United States,NC,No,No,No,NaN,100-500,...,Yes,No,No,No,No,No,No,No,NaN,North Carolina


# Data Cleansing

In [4]:
# Cleaning Gender Column

female_list = ['Female', 'female', 'Trans-female', 'Cis Female', 'F', 'Woman', 'f', 
 'Femake', 'woman', 'cis-female/femme', 'Trans woman', 'Female (trans)', 'Female (cis)', 'femail', "Female "]

male_list = ['M', 'Male', 'male', 'm', 'Male-ish', 'maile', 'Cis Male', 'Mal', 'Male (CIS)', 'Make',
       'Male ', 'Man', 'Mail', 'cis male', 'Malr', 'Cis Man', 'msle']

other_list = ['queer/she/they', 'non-binary', 'Nah', 'All', 'Enby', 'fluid', 'Genderqueer', 'Androgyne', 'Agender', 'something kinda male?', 'Guy (-ish) ^_^',
        'male leaning androgynous', 'Neuter', 'queer', 'A little about you', 'ostensibly male, unsure what that really means', 'p']

data['Gender'] = data['Gender'].replace(female_list, "Female")
data['Gender'] = data['Gender'].replace(male_list, "Male")
data['Gender'] = data['Gender'].replace(other_list, "Other")

data['Gender'].unique()

array(['Female', 'Male', 'Other'], dtype=object)

In [5]:
# Dropping Age Out of Bounds Column
data.drop(data[data['Age'] > 99].index, inplace = True)
data.drop(data[data['Age'] < 17].index, inplace = True)


In [6]:
# Only keeping United States data
data.drop(data[(data['Country'] != 'United States')].index, inplace = True)

In [7]:
#Creating new binary column for has_condition using work_interfere column

def label_hascondition (df):
    if df['work_interfere'] == 'NA' :
        return 0
    if df['work_interfere'] == 'Never' :
        return 1
    if df['work_interfere'] == 'Sometimes' :
        return 1
    if df['work_interfere'] == 'Often' :
        return 1
    if df['work_interfere'] == 'Rarely' :
        return 1
    else:
        return 0

data['has_condition'] = data.apply(lambda x: label_hascondition(x), axis=1)

In [8]:
#Creating StateID column for map graphic

def label_stateID (df):
    if df['state'] == 'AL' :
        return '1'
    if df['state'] == 'AK' :
        return 2
    if df['state'] == 'AZ' :
        return 3
    if df['state'] == 'AR' :
        return 4
    if df['state'] == 'CA' :
        return 5
    if df['state'] == 'CO' :
        return 6
    if df['state'] == 'CT' :
        return 7
    if df['state'] == 'DE' :
        return 8
    if df['state'] == 'FL' :
        return 9
    if df['state'] == 'GA' :
        return 10
    if df['state'] == 'HI' :
        return 11
    if df['state'] == 'ID' :
        return 12
    if df['state'] == 'IL' :
        return 13
    if df['state'] == 'IN' :
        return 14
    if df['state'] == 'IA' :
        return 15
    if df['state'] == 'KS' :
        return 16
    if df['state'] == 'KY' :
        return 17
    if df['state'] == 'LA' :
        return 18
    if df['state'] == 'ME' :
        return 19
    if df['state'] == 'MD' :
        return 20
    if df['state'] == 'MA' :
        return 21
    if df['state'] == 'MI' :
        return 22
    if df['state'] == 'MN' :
        return 23
    if df['state'] == 'MS' :
        return 24
    if df['state'] == 'MO' :
        return 25
    if df['state'] == 'MT' :
        return 26
    if df['state'] == 'NE' :
        return 27
    if df['state'] == 'NV' :
        return 28
    if df['state'] == 'NH' :
        return 29
    if df['state'] == 'NJ' :
        return 30
    if df['state'] == 'NM' :
        return 31
    if df['state'] == 'NY' :
        return 32
    if df['state'] == 'NC' :
        return 33
    if df['state'] == 'ND' :
        return 34
    if df['state'] == 'OH' :
        return 35
    if df['state'] == 'OK' :
        return 36
    if df['state'] == 'OR' :
        return 37
    if df['state'] == 'PA' :
        return 38
    if df['state'] == 'RI' :
        return 39
    if df['state'] == 'SC' :
        return 40
    if df['state'] == 'SD' :
        return 41
    if df['state'] == 'TN' :
        return 42
    if df['state'] == 'TX' :
        return 43
    if df['state'] == 'UT' :
        return 44
    if df['state'] == 'VT' :
        return 45
    if df['state'] == 'VA' :
        return 46
    if df['state'] == 'WA' :
        return 47
    if df['state'] == 'WV' :
        return 48
    if df['state'] == 'WI' :
        return 49
    if df['state'] == 'WY' :
        return 50



In [9]:
data['id'] = data.apply(lambda x: label_stateID(x), axis=1)
# df_states = data[['state_fullname', 'state']].drop_duplicates(subset=['state_fullname','state']).dropna()
# df_states = df_states.sort_values(by='state_fullname')
# df_states

In [10]:
data = data[(data.state.notnull())]


In [11]:
data

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments,state_fullname,has_condition,id
0,2014-08-27 11:29,37,Female,United States,IL,NaN,No,Yes,Often,25-Jun,...,Some of them,Yes,No,Maybe,Yes,No,NaN,Illinois,1,13
1,2014-08-27 11:29,44,Male,United States,IN,NaN,No,No,Rarely,More than 1000,...,No,No,No,No,Don't know,No,NaN,Indiana,1,14
4,2014-08-27 11:30,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Some of them,Yes,Yes,Yes,Don't know,No,NaN,Texas,1,43
5,2014-08-27 11:31,33,Male,United States,TN,NaN,Yes,No,Sometimes,25-Jun,...,Yes,Yes,No,Maybe,Don't know,No,NaN,Tennessee,1,42
6,2014-08-27 11:31,35,Female,United States,MI,NaN,Yes,Yes,Sometimes,05-Jan,...,Some of them,No,No,No,Don't know,No,NaN,Michigan,1,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,2015-08-25 19:59,36,Male,United States,UT,No,Yes,No,Rarely,More than 1000,...,Some of them,Some of them,No,No,Don't know,No,NaN,Utah,1,44
1255,2015-09-26 1:07,32,Male,United States,IL,No,Yes,Yes,Often,26-100,...,Some of them,Yes,No,No,Yes,No,NaN,Illinois,1,13
1256,2015-11-07 12:36,34,Male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,...,No,No,No,No,No,No,NaN,California,1,5
1257,2015-11-30 21:25,46,Female,United States,NC,No,No,No,NaN,100-500,...,No,No,No,No,No,No,NaN,North Carolina,0,33


In [12]:
data['self_employed'] = data['self_employed'].fillna("N/A")

<ipython-input-12-935544763d90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['self_employed'] = data['self_employed'].fillna("N/A")


# Sample Vizzes

In [13]:
# Does your employer provide mental health benefits?
alt.Chart(data).mark_bar().encode(
    x = alt.X('count()'),
    y = alt.Y('benefits', sort = '-x'))

alt.Chart(...)

In [14]:
alt.Chart(data).mark_bar().encode(
    x = alt.X('count()'),
    y = alt.Y('Country', sort = '-x'))

alt.Chart(...)

In [15]:
data.supervisor.value_counts(normalize=True)

Yes             0.408163
No              0.318367
Some of them    0.273469
Name: supervisor, dtype: float64

In [16]:
data.Gender.value_counts(normalize=True)

Male      0.746939
Female    0.247619
Other     0.005442
Name: Gender, dtype: float64

# Export to Processed folder

In [19]:
data.to_csv('../data/processed/processed_survey.csv')